In [60]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [61]:
import pandas as pd
import numpy as np

df_pesoEaltura = pd.read_csv("/content/drive/MyDrive/Data/BDIPMamaV11-INTELIPesoEAltura_DATA_LABELS_2023-01-24_1926.csv")

In [62]:
df_pesoEaltura = df_pesoEaltura.rename(columns={"Record ID": "record_id",
                                                "Data: ": "data",
                                                "Peso": "peso",
                                                "Altura (em centimetros)": "altura"})

lista_RecorId = df_pesoEaltura['record_id'].unique()
df_novo = pd.DataFrame()

for i in lista_RecorId: 
    df_query = df_pesoEaltura.query(f'record_id == {i}')
    df_query = df_query.sort_values(by='data')

    df_row_first = df_query.iloc[0]

    if len(df_query) > 1: 
      df_row_second = df_query.iloc[1]

      numero = len(df_query) - 1

      for col in ['altura', 'peso', 'IMC']:
        while (not pd.isna(df_row_second[col].all())) and (not pd.isna(df_row_second['data'])) and (numero > 1):
          df_row_second = df_query.iloc[len(df_query) - numero]
          numero -= 1

      df_row_first['ultimo_peso'] = df_row_second['peso']
      df_row_first['ultima_altura'] = df_row_second['altura']
      df_row_first['ultimo_IMC'] = df_row_second['IMC']
      df_row_first['ultima_data'] = df_row_second['data']
      
      df_novo = df_novo.append(df_row_first, ignore_index=True)
      
    else: 
      df_novo = df_novo.append(df_row_first, ignore_index=True)

df_novo = df_novo.rename(columns={
                        "peso": "primeiro_peso", 
                        "altura": "primeira_altura",
                        "IMC": "primeiro_IMC"
                        })

df_pesoEaltura = df_novo

/usr/local/lib/python3.8/dist-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()
/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1724: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer_missing(indexer, value)


In [63]:
np.random.seed(73)

df_pesoEaltura['primeiro_IMC'] = (df_pesoEaltura['primeiro_peso'] / pow(df_pesoEaltura['primeira_altura']/100, 2))

lista_pesoEaltura = ["primeiro_peso", "primeira_altura", "ultimo_peso", "ultima_altura", "primeiro_IMC"]
lista_normal = []

for i in lista_pesoEaltura: 
  df_pesoEaltura[i] = df_pesoEaltura[i].fillna(pd.Series(np.random.normal(df_pesoEaltura[i].std(skipna=True), 
                                                                          df_pesoEaltura[i].std(skipna=True), 
                                                                          df_pesoEaltura[i].isnull().sum()), 
                                                         index=df_pesoEaltura[i].index[df_pesoEaltura[i].isnull()]))

df_pesoEaltura['ultimo_IMC'] = (df_pesoEaltura['ultimo_peso'] / pow(df_pesoEaltura['ultima_altura']/100, 2))

df_pesoEaltura

,record_id,Repeat Instrument,Repeat Instance,data,primeiro_peso,primeira_altura,primeiro_IMC,ultimo_peso,ultima_altura,ultimo_IMC,ultima_data
0,302.0,Dados Antropometricos,20.0,2009-01-02,57.80,158.0,23.153341,64.900000,158.0,25.997436,2013-05-23
1,710.0,Dados Antropometricos,18.0,2008-07-12,61.00,156.0,25.065746,63.800000,155.0,26.555671,2020-10-31
2,752.0,Dados Antropometricos,2.0,2009-12-03,84.00,152.0,36.357341,85.900000,152.0,37.179709,2013-11-27
3,1367.0,Dados Antropometricos,1.0,2010-04-28,57.90,143.0,28.314343,51.000000,143.0,24.940095,2011-07-27
4,1589.0,Dados Antropometricos,12.0,2009-04-03,63.20,167.0,22.661264,63.650000,167.0,22.822618,2013-09-05
...,...,...,...,...,...,...,...,...,...,...,...
4267,82123.0,Dados Antropometricos,5.0,2021-02-07,85.40,153.0,36.481695,24.018708,154.0,10.127639,2021-10-25
4268,82124.0,Dados Antropometricos,1.0,2020-11-10,81.10,151.0,35.568615,78.000000,151.0,34.209026,2020-12-05
4269,82131.0,Dados Antropometricos,1.0,2020-11-01,71.40,156.5,29.152079,75.650000,156.0,31.085634,2021-01-04
4270,82205.0,Dados Antropometricos,4.0,2021-02-28,135.85,174.0,44.870525,23.936242,174.0,7.906012,2021-08-24


In [64]:
df_pesoEaltura['diferenca_peso'] = df_pesoEaltura['ultimo_peso'] - df_pesoEaltura['primeiro_peso']
df_pesoEaltura['coef_peso'] = df_pesoEaltura['ultimo_peso'] / df_pesoEaltura['primeiro_peso']

df_pesoEaltura['ultima_data'] = pd.to_datetime(df_pesoEaltura['ultima_data'])
df_pesoEaltura['data'] = pd.to_datetime(df_pesoEaltura['data'])

df_pesoEaltura['diferenca_altura'] = df_pesoEaltura['ultima_data'] - df_pesoEaltura['data']

df_pesoEaltura['diferenca_tempo'] = (df_pesoEaltura['ultima_data'] - df_pesoEaltura['data']).dt.days
df_pesoEaltura['coef_peso_tempo'] = df_pesoEaltura['diferenca_peso'] / df_pesoEaltura['diferenca_tempo']

df_pesoEaltura['Repeat Instrument'] = df_pesoEaltura['Repeat Instrument'].fillna(0)
df_pesoEaltura['Repeat Instance'] = df_pesoEaltura['Repeat Instance'].fillna(0)
df_pesoEaltura['data'] = df_pesoEaltura['data'].fillna(0)
df_pesoEaltura['ultima_data'] = df_pesoEaltura['ultima_data'].fillna(0)
df_pesoEaltura['diferenca_altura'] = df_pesoEaltura['diferenca_altura'].fillna(pd.Timedelta(0))

df_pesoEaltura['diferenca_tempo'] = df_pesoEaltura['diferenca_tempo'].fillna(0)
df_pesoEaltura['coef_peso_tempo'] = df_pesoEaltura['coef_peso_tempo'].fillna(0)

,record_id,Repeat Instrument,Repeat Instance,data,primeiro_peso,primeira_altura,primeiro_IMC,ultimo_peso,ultima_altura,ultimo_IMC,ultima_data,diferenca_peso,coef_peso,diferenca_altura,diferenca_tempo,coef_peso_tempo
0,302.0,Dados Antropometricos,20.0,2009-01-02 00:00:00,57.80,158.0,23.153341,64.900000,158.0,25.997436,2013-05-23 00:00:00,7.100000,1.122837,1602 days,1602.0,0.004432
1,710.0,Dados Antropometricos,18.0,2008-07-12 00:00:00,61.00,156.0,25.065746,63.800000,155.0,26.555671,2020-10-31 00:00:00,2.800000,1.045902,4494 days,4494.0,0.000623
2,752.0,Dados Antropometricos,2.0,2009-12-03 00:00:00,84.00,152.0,36.357341,85.900000,152.0,37.179709,2013-11-27 00:00:00,1.900000,1.022619,1455 days,1455.0,0.001306
3,1367.0,Dados Antropometricos,1.0,2010-04-28 00:00:00,57.90,143.0,28.314343,51.000000,143.0,24.940095,2011-07-27 00:00:00,-6.900000,0.880829,455 days,455.0,-0.015165
4,1589.0,Dados Antropometricos,12.0,2009-04-03 00:00:00,63.20,167.0,22.661264,63.650000,167.0,22.822618,2013-09-05 00:00:00,0.450000,1.007120,1616 days,1616.0,0.000278
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4267,82123.0,Dados Antropometricos,5.0,2021-02-07 00:00:00,85.40,153.0,36.481695,24.018708,154.0,10.127639,2021-10-25 00:00:00,-61.381292,0.281250,260 days,260.0,-0.236082
4268,82124.0,Dados Antropometricos,1.0,2020-11-10 00:00:00,81.10,151.0,35.568615,78.000000,151.0,34.209026,2020-12-05 00:00:00,-3.100000,0.961776,25 days,25.0,-0.124000
4269,82131.0,Dados Antropometricos,1.0,2020-11-01 00:00:00,71.40,156.5,29.152079,75.650000,156.0,31.085634,2021-01-04 00:00:00,4.250000,1.059524,64 days,64.0,0.066406
4270,82205.0,Dados Antropometricos,4.0,2021-02-28 00:00:00,135.85,174.0,44.870525,23.936242,174.0,7.906012,2021-08-24 00:00:00,-111.913758,0.176196,177 days,177.0,-0.632281


In [ ]:
df_pesoEaltura